# Downloading the Data from Kaggle

In [ ]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Found existing installation: kaggle 1.5.6
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.3)
Processing /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01/kaggle-1.5.6-py3-none-any.whl


In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c plant-seedlings-classification

 99% 1.67G/1.69G [00:23<00:00, 80.1MB/s]
100% 1.69G/1.69G [00:23<00:00, 77.1MB/s]


In [ ]:
! mkdir plant-seedlings-classification
! unzip ./plant-seedlings-classification.zip -d plant-seedlings-classification

# 1) Data Loading and Preparation

In [ ]:
import numpy as np
import pandas as pd
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
data_dir = './plant-seedlings-classification/'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
sample_submission = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (270, 270)

In [ ]:
print("Train Dataset")
train_dataset  = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  shuffle=True,
  image_size= IMG_SIZE,
  batch_size= BATCH_SIZE)

print("Validation Dataset")
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  shuffle=True,
  image_size= IMG_SIZE,
  batch_size=BATCH_SIZE)


Train Dataset
Found 4750 files belonging to 12 classes.
Using 3800 files for training.
Validation Dataset
Found 4750 files belonging to 12 classes.
Using 950 files for validation.


In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

Number of validation batches: 24
Number of test batches: 6


In [ ]:
class_names = train_dataset.class_names
num_classes = len(class_names)
print(class_names)

['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_ds = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_ds = test_dataset.prefetch(buffer_size=AUTOTUNE)

# 2) Data Agumentation

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

# 3) Load Models

In [ ]:
vgg_model_file = 'drive/My Drive/week/extract_VGG16'
inception_model_file = 'drive/My Drive/week/extract_InceptionV3'
resnet_model_file = 'drive/My Drive/week/extract_Resnet50'

In [ ]:
vgg_model = tf.keras.models.load_model(vgg_model_file)
inception_model = tf.keras.models.load_model(inception_model_file)
resnet_model = tf.keras.models.load_model(resnet_model_file)

# 4) TF-TRT

In [ ]:
import time
from tensorflow.python.compiler.tensorrt import trt_convert as trt

In [ ]:
for data in test_ds.take(1):
  test_images = data[0]
  test_labels = data[1]

## Some Helper Functions

In [ ]:
def input_map_fn():
    for data in test_ds.take(1):
      test_images = data[0]
      yield np.array([test_images])

In [ ]:
def time_trt_model(concrete_func):
    times = []
    for i in range(500):
        start_time = time.time()
        one_prediction = concrete_func(tf.constant(test_images))
        delta = (time.time() - start_time)
        times.append(delta)
    mean_delta = np.array(times).mean()
    fps = 1 / mean_delta
    print('\n average(sec):{:.2f},fps:{:.2f}'.format(mean_delta, fps))

In [ ]:
def trtPredict(concrete_func):
  for data in test_ds.take(1):
    test_images = data[0]
    test_labels = data[1]

  output = concrete_func(tf.constant(test_images))
  for key in output:
    precited_labels = np.argmax(output[key].numpy(), axis=1)

  accuracy = np.mean(precited_labels == test_labels)
  print('\nTest accuracy:{:.2f}'.format(accuracy))

## Functions for FP16, FP32 and INT8

In [ ]:
def wholeTRT_FP16(savefile):

  params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(precision_mode=trt.TrtPrecisionMode.FP16)

  converter = trt.TrtGraphConverterV2(
      input_saved_model_dir=savefile,
      conversion_params=params)
  converter.convert()

  saved_model_dir_trt = savefile[20:] + '_FP16.trt'

  converter.save(saved_model_dir_trt)

  root = tf.saved_model.load(saved_model_dir_trt)
  concrete_func = root.signatures['serving_default']

  trtPredict(concrete_func)

  time_trt_model(concrete_func)

  print('\n===============================\n')

def wholeTRT_FP32(savefile):

  params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(precision_mode=trt.TrtPrecisionMode.FP32)

  converter = trt.TrtGraphConverterV2(
      input_saved_model_dir=savefile,
      conversion_params=params)
  converter.convert()

  saved_model_dir_trt = savefile[20:] + '_FP32.trt'

  converter.save(saved_model_dir_trt)

  root = tf.saved_model.load(saved_model_dir_trt)
  concrete_func = root.signatures['serving_default']

  trtPredict(concrete_func)

  time_trt_model(concrete_func)

  print('\n===============================\n')

def wholeTRT_INT8(savefile):
  params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(
    precision_mode= trt.TrtPrecisionMode.INT8,
    use_calibration=True)

  converter = trt.TrtGraphConverterV2(
      input_saved_model_dir=savefile,
      conversion_params=params)
  
  converter.convert(calibration_input_fn=input_map_fn)  

  saved_model_dir_trt = savefile[20:] + '_INT8.trt'

  converter.save(saved_model_dir_trt)

  root = tf.saved_model.load(saved_model_dir_trt)
  concrete_func = root.signatures['serving_default']

  trtPredict(concrete_func)

  time_trt_model(concrete_func)

  print('\n===============================\n')

### 1) FP16

In [ ]:
wholeTRT_FP16(vgg_model_file)
wholeTRT_FP16(inception_model_file)
wholeTRT_FP16(resnet_model_file)

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Assets written to: extract_VGG16_FP16.trt/assets

Test accuracy:0.84

 average(sec):0.09,fps:11.51


INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Assets written to: extract_InceptionV3_FP16.trt/assets

Test accuracy:0.72

 average(sec):0.06,fps:16.54


INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Assets written to: extract_Resnet50_FP16.trt/assets

Test accuracy:0.84

 average(sec):0.08,fps:12.88




### 2) FP32

In [ ]:
wholeTRT_FP32(vgg_model_file)
wholeTRT_FP32(inception_model_file)
wholeTRT_FP32(resnet_model_file)

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Assets written to: extract_VGG16_FP32.trt/assets

Test accuracy:0.91

 average(sec):0.09,fps:11.52


INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Assets written to: extract_InceptionV3_FP32.trt/assets

Test accuracy:0.91

 average(sec):0.06,fps:16.55


INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Assets written to: extract_Resnet50_FP32.trt/assets

Test accuracy:0.97

 average(sec):0.08,fps:12.88




### 3) INT8

In [ ]:
wholeTRT_INT8(vgg_model_file)

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Assets written to: extract_VGG16_INT8.trt/assets

Test accuracy:0.88

 average(sec):0.09,fps:11.52




In [ ]:
wholeTRT_INT8(inception_model_file)

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Assets written to: extract_InceptionV3_INT8.trt/assets

Test accuracy:0.88

 average(sec):0.06,fps:16.53




In [ ]:
wholeTRT_INT8(resnet_model_file)

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Assets written to: extract_Resnet50_INT8.trt/assets

Test accuracy:0.81

 average(sec):0.08,fps:12.88




# 5) TensorFlow Model Optimization Toolkit



In [ ]:
!pip install -q tensorflow-model-optimization

In [ ]:
import tempfile
import tensorflow_model_optimization as tfmot

## TF Lite Evaluator function

In [ ]:
def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  for data in validation_dataset.take(1):
    test_images = data[0]
    test_labels = data[1]

  # Run predictions on ever y image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images.numpy()):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels.numpy()).mean()
  return accuracy

### Helper function for file size

In [ ]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

# Pruning and Quantization function

In [ ]:
def pruned_and_qunatized(basemodel):
  model = basemodel

  model.compile(
    optimizer='RMSprop',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

  """
  Base Model
  """

  _, baseline_model_accuracy = model.evaluate(test_ds, verbose=0)  
  print('Baseline test accuracy:', baseline_model_accuracy)

  _, keras_file = tempfile.mkstemp('.h5')
  tf.keras.models.save_model(model, keras_file, include_optimizer=False)
  print('Saved baseline model to:', keras_file)

  """
  Pruning Model
  """
  
  model_for_export = tfmot.sparsity.keras.strip_pruning(model)

  _, pruned_keras_file = tempfile.mkstemp('.h5')
  tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
  print('Saved pruned Keras model to:', pruned_keras_file)


  print("\nSize of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
  print("Size of gzipped pruned Keras model: %.2f bytes\n" % (get_gzipped_model_size(pruned_keras_file)))

  """
  Qunatized Model
  """

  converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  quantized_and_pruned_tflite_model = converter.convert()

  _, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

  with open(quantized_and_pruned_tflite_file, 'wb') as f:
    f.write(quantized_and_pruned_tflite_model)

  print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

  print("\nSize of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
  print("Size of gzipped pruned and quantized TFlite model: %.2f bytes\n" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))


  interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
  interpreter.allocate_tensors()

  test_accuracy = evaluate_model(interpreter)

  print('Pruned and quantized test_accuracy:', test_accuracy)
  print('Baseline test accuracy:', baseline_model_accuracy)

## 1) VGG16

In [ ]:
pruned_and_qunatized(vgg_model)

Baseline test accuracy: 0.8645833134651184
Saved baseline model to: /tmp/tmp4z1zpl_t.h5
Saved pruned Keras model to: /tmp/tmp_tvvej1i.h5

Size of gzipped baseline Keras model: 55227288.00 bytes
Size of gzipped pruned Keras model: 55227288.00 bytes

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tmpjjk5gge5/assets
Saved quantized and pruned TFLite model to: /tmp/tmpiq0vztgr.tflite

Size of gzipped baseline Keras model: 55227288.00 bytes
Size of gzipped pruned and quantized TFlite model: 8636002.00 bytes

Evaluated on 0 results so far.


Pruned and quantized test_accuracy: 0.90625
Baseline test accuracy: 0.8645833134651184


## 2) INCEPTION V4

In [ ]:
pruned_and_qunatized(inception_model)

Baseline test accuracy: 0.8229166865348816
Saved baseline model to: /tmp/tmpyjg3yicy.h5
Saved pruned Keras model to: /tmp/tmp2n8p2p74.h5

Size of gzipped baseline Keras model: 82962682.00 bytes
Size of gzipped pruned Keras model: 82962678.00 bytes

INFO:tensorflow:Assets written to: /tmp/tmp6vlexxw8/assets


INFO:tensorflow:Assets written to: /tmp/tmp6vlexxw8/assets


Saved quantized and pruned TFLite model to: /tmp/tmpxmn_1jo2.tflite

Size of gzipped baseline Keras model: 82962682.00 bytes
Size of gzipped pruned and quantized TFlite model: 15370417.00 bytes

Evaluated on 0 results so far.


Pruned and quantized test_accuracy: 0.75
Baseline test accuracy: 0.8229166865348816


##3) RESNET50

In [ ]:
pruned_and_qunatized(resnet_model)

Baseline test accuracy: 0.8802083134651184
Saved baseline model to: /tmp/tmpa_si6wjx.h5
Saved pruned Keras model to: /tmp/tmp6thalmbq.h5

Size of gzipped baseline Keras model: 89472862.00 bytes
Size of gzipped pruned Keras model: 89472859.00 bytes

INFO:tensorflow:Assets written to: /tmp/tmpy9m15v0o/assets


INFO:tensorflow:Assets written to: /tmp/tmpy9m15v0o/assets


Saved quantized and pruned TFLite model to: /tmp/tmpo8v8aszr.tflite

Size of gzipped baseline Keras model: 89472862.00 bytes
Size of gzipped pruned and quantized TFlite model: 15317161.00 bytes

Evaluated on 0 results so far.


Pruned and quantized test_accuracy: 1.0
Baseline test accuracy: 0.8802083134651184
